In [31]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING"]="true"

In [32]:
from langchain_groq import ChatGroq

llm= ChatGroq(model="llama3-8b-8192")

In [33]:
resp=llm.invoke("How many districts are there in Tamilnadu?")
resp

AIMessage(content='There are 38 districts in the Indian state of Tamil Nadu.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 20, 'total_tokens': 34, 'completion_time': 0.011666667, 'prompt_time': 0.004862134, 'queue_time': 0.009059646000000001, 'total_time': 0.016528801}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-a5a138d4-dd15-4f25-b8ba-5c842fa80b73-0', usage_metadata={'input_tokens': 20, 'output_tokens': 14, 'total_tokens': 34})

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","You are an expert chef knowing various cuisines and who is so enthusiastic and curious.You have knowledge of different cuisines around the world yet you are so addicted to {cuisine_type} cuisine.You feel it is both healthy,variety and tasty .Answer the questions based on the relevance of given context and wrap the output in `json` tags \n{format_instructions}"),
    ("human","Hello chef!Great to have conversation with you"),
    ("ai","Voila.Nice to meet you.Happy to answer your questions."),
    ("human","I know {level} level of cooking.I would be happy if you could explain me based on my level of knowledge and skill."),
    ("ai","Sure..I would explain you based on your level!!"),
    ("user","{input}")
])


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()
chain=prompt | llm | output_parser
chat_response=chain.invoke({"input":"Which cuisine is the best one in world and why?","cuisine_type":"French","level":"expert"})
chat_response


In [39]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel,Field
from typing import List

class ChefOutputFormat(BaseModel):
    answer:str=Field(...,description="Short answer to the question asked")
    reason:str=Field(...,description="Detailed explanation or reason for the short answer provided")
    reference:List[str]=Field(...,description="Reference from other resources to the answer provided if reference is needed.")

parser=PydanticOutputParser(pydantic_object=ChefOutputFormat)
    

In [40]:
prompt=ChatPromptTemplate.from_messages([
    ("system","You are an expert chef knowing various cuisines and who is so enthusiastic and curious.You have knowledge of different cuisines around the world yet you are so addicted to {cuisine_type} cuisine.You feel it is both healthy,variety and tasty .Answer the questions based on the relevance of given context.Wrap the output in ```json tags\n{format_instructions}\n```"),
    ("human","Hello chef!Great to have conversation with you"),
    ("ai","Voila.Nice to meet you.Happy to answer your questions."),
    ("human","I know {level} of cooking.I would be happy if you could explain me based on my level of knowledge and skill."),
    ("ai","Sure..I would explain you based on your level!!"),
    ("user","{input}")
]).partial(format_instructions=parser.get_format_instructions())

In [41]:
chain=prompt | llm | parser
chat_response=chain.invoke({"input":"Which cuisine is the best one in world and why?","cuisine_type":"French","level":"expert"})
chat_response

ChefOutputFormat(answer='French cuisine', reason='French cuisine is a masterpiece of culinary art, offering a harmonious balance of flavors, textures, and presentation. With its rich history and cultural heritage, French cuisine has evolved over centuries to become a benchmark for fine dining. The attention to detail, the use of fresh and high-quality ingredients, and the technique-driven approach make French cuisine both healthy and delicious. From the rich flavors of Bouillabaisse to the delicate pastries of Macarons, French cuisine is a symphony of flavors and textures that never fails to impress.', reference=['Michelin-starred restaurants', 'French culinary schools', 'French gastronomic tourism'])

In [44]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


context_relevant_prompt=ChatPromptTemplate.from_template(
    """
    Answer the questions basd on the current context 
    <context>
    {context}
    </context>
    """
)


In [45]:
document_chain=create_stuff_documents_chain(llm,context_relevant_prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the questions basd on the current context \n    <context>\n    {context}\n    </context>\n    '))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x16139b310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x127f24a90>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})